In [ ]:
import pandas as pd    
import glob
from tqdm import tqdm

In [ ]:
ARXIV_DUMP_PATH = '../data/arxiv_dump/search_results.json'

S2_ORC_INPUT_PATHS = '../data/s2orc/metadata/comp_sci/*.jsonl'

CITATIONS_OUTPUT_PATH = 'citation_papers.csv'
MERGED_OUTPUT_PATH = 'arxiv_s2orc_matched.csv'

In [ ]:
s2orc_paths = glob.glob(S2_ORC_INPUT_PATHS)
len(s2orc_paths)

In [ ]:
s2orc_papers = pd.DataFrame()

for path in tqdm(s2orc_paths):
    df2 = pd.read_json(path, lines=True)
    s2orc_papers = s2orc_papers.append(df2[~df2['arxiv_id'].isna()])

In [ ]:
arxiv_papers = pd.read_json(path_or_buf=ARXIV_DUMP_PATH, orient='index')

In [ ]:
print("Has arxiv id:", len(s2orc_papers))
#print("Has arxiv id:",sum(~s2orc_papers.arxiv_id.isna()))

# Papers in arxiv dump covered in S2ORC

## By arxiv id

In [ ]:
#removing 'vXX' suffix
arxiv_papers['arxiv_id'] = arxiv_papers.id.str.split('v').str[0]

In [ ]:
print("All:", len(arxiv_papers))
print("Matched id:", sum(s2orc_papers['arxiv_id'].isin(arxiv_papers['arxiv_id'])))

In [ ]:
merged = pd.merge(left=arxiv_papers, right=s2orc_papers, on='arxiv_id')

In [ ]:
merged.columns

In [ ]:
merged.to_csv(MERGED_OUTPUT_PATH)

In [ ]:
merged.year.hist()

## Citations

In [ ]:
all_citations_ids = set()
for citations in merged.outbound_citations:
    all_citations_ids |= set(citations)

In [ ]:
len(all_citations_ids)

In [ ]:
citation_papers = pd.DataFrame()

for path in tqdm(s2orc_paths):
    df2 = pd.read_json(path, lines=True)
    citation_papers = citation_papers.append(df2[df2.paper_id.isin(all_citations_ids)])

In [ ]:
citation_papers.to_csv(CITATIONS_OUTPUT_PATH)

In [ ]:
citation_papers.year.hist()